In [40]:
CLIENT_ID = 'xxx' # your Foursquare ID

CLIENT_SECRET = 'xxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Your credentails:


In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

import json 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
html_adr = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_adr

r = requests.get(html_adr)
soup = BeautifulSoup(r.content,  'html.parser')

# extracting data with bs

In [4]:
# soup.find_all('td')
My_table = soup.find('table',{'class':'wikitable sortable'})
td_all = My_table.find_all('td')

# good solution -> to list

In [6]:
print('--good solution -> to list - ---------------------')  
lista = []    
i=0;
for element in td_all:
    try:
        lista.append(element.find('a').string.strip()) 
    except:
        lista.append(element.string.strip())
    finally:
        i+=1
        
print('liczba iteracji=', i)
lista[:12]

--good solution -> to list - ---------------------
liczba iteracji= 867


['M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A',
 'North York',
 'Parkwoods',
 'M4A',
 'North York',
 'Victoria Village']

In [7]:
# PostalCode, Borough, Neighborhood

df = pd.DataFrame(np.array(lista).reshape(-1,3), columns = ['PostalCode', 'Borough', 'Neighborhood'])
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [8]:
df.tail(10)

,PostalCode,Borough,Neighborhood
279,M4Z,Not assigned,Not assigned
280,M5Z,Not assigned,Not assigned
281,M6Z,Not assigned,Not assigned
282,M7Z,Not assigned,Not assigned
283,M8Z,Etobicoke,Kingsway Park South West
284,M8Z,Etobicoke,Mimico NW
285,M8Z,Etobicoke,The Queensway West
286,M8Z,Etobicoke,Royal York South West
287,M8Z,Etobicoke,South of Bloor
288,M9Z,Not assigned,Not assigned


In [11]:
# df.to_csv('List_of_postal_codes_of_Canada_scrapped_from_wiki.csv')
# df = pd.read_csv('List_of_postal_codes_of_Canada_scrapped_from_wiki.csv') # gdy laduje juz zescrapowane
df = df[['PostalCode', 'Borough', 'Neighborhood']]
print(df.index)
print(df.shape)
print(df.columns)
df.head(10)


RangeIndex(start=0, stop=289, step=1)
(289, 3)
Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


"...process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. "

In [9]:
df_ok = df[df['Borough'] != 'Not assigned']
df_ok.reset_index(inplace=True, drop=True)
print('SHAPE: ', df_ok.shape)
df_ok.head(10)

SHAPE:  (212, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


"More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table. "

In [10]:
df_agg = df_ok.groupby(['PostalCode','Borough'], as_index=False, sort=False).agg( ','.join) # BEZ >>as_index=False<< zle, bo 1 kolumna...
print(df_agg.columns)
print(df_agg.index)
print(df_agg.shape)
df_agg.head(15)

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')
RangeIndex(start=0, stop=103, step=1)
(103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


"If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park. """

In [11]:
df_agg2 = df_agg
print(df_agg2.index)
df_agg2.columns

RangeIndex(start=0, stop=103, step=1)


Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

In [12]:
df_agg2['new_col'] = df_agg2[['Borough','Neighborhood']].apply(lambda x: x['Borough'] if x['Neighborhood']=='Not assigned' else x['Neighborhood'], axis=1)  
df_agg2 = df_agg2[['PostalCode', 'Borough', 'new_col']]
df_agg2.columns = ['PostalCode', 'Borough', 'Neighborhood']
df_agg2.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [13]:
df_agg2.shape

(103, 3)

# ---------------------  end of part # 1 of assignment  ----------------------------

In [ ]:
# import geocoder 
# lat_lng_coords = None

# loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

In [18]:
# Given that this package can be very unreliable, in case you are not able to get the geographical coordinates 
# of the neighborhoods using the Geocoder package, here is a link to a csv file 
df_coord = pd.read_csv('Geospatial_Coordinates.csv')
print(df_coord.shape)
df_coord.columns = ['PostalCode', 'Latitude', 'Longitude']
df_coord.head(10)

(103, 3)


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [24]:
df_join = df_agg2.join(df_coord.set_index('PostalCode'), on='PostalCode', how='left', rsuffix='_')
print(df_join.shape)
df_join.head(15)

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.



In [29]:
print(df_join.iloc[2][1])
print(df_join.loc[2]['Borough'])
df_join.loc[2]['Borough'].find('Toronto')

Downtown Toronto
Downtown Toronto


9

In [33]:
df_join2 = df_join[df_join['Borough'].str.find('Toronto') >=0]
print(df_join2.shape)
df_join2.head(10)

(38, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
31,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259
36,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752


# ---- end of part # 2 -------------